<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/evalute-BERTScore-bert-paradetox-1Token-split-masked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install transformers datasets
!pip install transformers[torch]
!pip install bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 2.0 MB/s eta 0:00:00


In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForMaskedLM
from bert_score import score
from datasets import load_dataset

# Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("HamdanXI/bert-paradetox-1Token-split-masked")
model = AutoModelForMaskedLM.from_pretrained("HamdanXI/bert-paradetox-1Token-split-masked")

# Load your dataset
dataset = load_dataset("HamdanXI/paradetox-1Token-Split")  # Replace with the path to your dataset

# Function to predict the masked word
def predict_masked_word(masked_text):
    inputs = tokenizer(masked_text, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    masked_index = (inputs.input_ids[0] == tokenizer.mask_token_id).nonzero(as_tuple=True)[0]
    predicted_token_id = logits[0, masked_index, :].argmax(axis=-1)
    predicted_token = tokenizer.decode(predicted_token_id)
    return masked_text.replace(tokenizer.mask_token, predicted_token)

# Generate predictions and compute BERTScore
bert_scores = []
for i in range(len(dataset['test'])):  # Use 'validation' or 'test' as needed
    toxic_comment = dataset['test'][i]['en_toxic_comment']
    neutral_comment = dataset['test'][i]['en_neutral_comment']
    masked_comment = dataset['test'][i]['masked_comment']

    # Generate prediction
    prediction = predict_masked_word(masked_comment)

    # Compute BERTScore
    P, R, F1 = score([prediction], [neutral_comment], lang="en", verbose=True)
    bert_scores.append(F1.mean().item())

# Calculate the average BERTScore
average_bert_score = sum(bert_scores) / len(bert_scores)
print(f"Average BERTScore: {average_bert_score}")

In [14]:
print(f"Average BERTScore: {average_bert_score}")

Average BERTScore: 0.9379353561471923
